# NetCDF zu GeoTIFF Konvertierung
Dieses Skript lädt NetCDF-Dateien, extrahiert relevante Daten, und speichert sie als GeoTIFF-Dateien. Da NetCDF-Dateien oft Zeitschritte enthalten, wird das Skript die Daten über die Zeitachsen iterieren und mehrere TIFF-Dateien erzeugen (eines pro Zeitschritt).

### Bibliotheken:
- `xarray`: Für das Laden und Bearbeiten von NetCDF-Daten.
- `rasterio`: Für die Erstellung von GeoTIFF-Dateien.
- `numpy`: Für die Arbeit mit numerischen Daten.

### Vorgehen:
1. Bibliotheken instllieren, laden und NetCDF-Datei öffnen.
2. Überprüfen der Datenstruktur (Dimensionen, Variablen).
3. Konvertierung der NetCDF-Daten (pro Zeitschritt) in GeoTIFF.


In [1]:
# Funktion zur Prüfung und Installation von Modulen
import sys
import subprocess

def install_and_import(package):
    try:
        __import__(package)
    except ImportError:
        print(f"{package} ist nicht installiert. Installation wird durchgeführt...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    finally:
        globals()[package] = __import__(package)

# Liste der benötigten Module
modules = ['xarray', 'rasterio', 'numpy', 'os', 'packaging', 'pandas', 'netCDF4', 'h5netcdf', 'h5py', 'scipy']

# Prüfen und ggf. Installieren der Module
for module in modules:
    install_and_import(module)


In [2]:
# Bibliotheken importieren
import xarray as xr
import rasterio
import numpy as np
import os
import packaging
import pandas
import netCDF4
import h5netcdf
import h5py
import scipy


## Schritt 1: NetCDF-Datei laden
Verwende die Bibliothek `xarray`, um die NetCDF-Datei zu öffnen und einen Überblick über die enthaltenen Daten und Dimensionen zu bekommen.
Bitte passe den Pfad zu deiner NetCDF-Datei an.


In [3]:
# NetCDF-Datei laden (Pfad zur Datei anpassen)
nc_file_path = './data/01/YW_2017.002_20210101.nc'

# Öffnen der NetCDF-Datei mit xarray
ds = xr.open_dataset(nc_file_path, engine='netcdf4')

# Überblick über die Datenstruktur (Dimensionen, Variablen)
print(ds)

# Wenn du die Zeitschritte oder andere Variablen sehen möchtest:
print("Zeitliche Dimension:", ds.coords['time'].values)


<xarray.Dataset> Size: 1GB
Dimensions:  (x: 900, y: 1100, time: 288)
Coordinates:
  * x        (x) float64 7kB -4.43e+05 -4.42e+05 -4.41e+05 ... 4.55e+05 4.56e+05
  * y        (y) float64 9kB -4.758e+06 -4.757e+06 ... -3.66e+06 -3.659e+06
    lat      (y, x) float64 8MB ...
    lon      (y, x) float64 8MB ...
  * time     (time) datetime64[ns] 2kB 2021-01-01 ... 2021-01-01T23:55:00
Data variables:
    crs      int32 4B ...
    RR       (time, y, x) float32 1GB ...
Attributes:
    title:        RADKLIM - radar-based precipitation climatology
    institution:  Deutscher Wetterdienst (DWD)
    history:      Created at 2024-04-15 09:58:17.534322
    reference:    10.5676/DWD/RADKLIM_YW_V2017.002
    Conventions:  CF-1.6
    Author:       Harald Rybka, Katharina Lengfeld
Zeitliche Dimension: ['2021-01-01T00:00:00.000000000' '2021-01-01T00:05:00.000000000'
 '2021-01-01T00:10:00.000000000' '2021-01-01T00:15:00.000000000'
 '2021-01-01T00:20:00.000000000' '2021-01-01T00:25:00.000000000'
 '2021-

In [6]:
ds['crs'].attrs

{'long_name': 'RADOLAN Grid',
 'grid_mapping_name': 'polar_stereographic',
 'latitude_of_projection_origin': 90.0,
 'straight_vertical_longitude_from_pole': 10.0,
 'semi_major_axis': 6370040.0,
 'semi_minor_axis': 6370040.0,
 'earth_radius': 6370040.0,
 'standard_parallel': 60.0,
 'false_easting': 0.0,
 'false_northing': 0.0,
 'inverse_flattening': 0.0,
 'scale_factor_at_projection_origin': 0.9330127019,
 'units': 'm',
 'proj4': '+proj=stere +lat_0=90 +lat_ts=90 +lon_0=10 +k_0=0.933012701892 +x_0=0 +y_0=0 +a=6370040 +b=6370040 +units=km +no_defs',
 'crs_wkt': 'PROJCRS["Stereographic_North_Pole",BASEGEOGCRS["GCS_unnamed ellipse",DATUM["D_unknown",ELLIPSOID["Unknown",6370040,0,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["Degree",0.0174532925199433]]],CONVERSION["unnamed",METHOD["Polar Stereographic (variant B)"],PARAMETER["Latitude of standard parallel",60,ANGLEUNIT["Degree",0.0174532925199433]],PARAMETER["Longitude of origin",10,ANGLEUNIT["Degree",0.0174532925199433]],PARAMET

In [7]:
# Extrahiere die 'crs'-Information und zeige sie an
if 'crs' in ds.variables:
    crs_value = ds['crs'].attrs.get('proj4', 'Keine gültige CRS-Information im proj4-Format vorhanden')
    print(f"CRS-Proj4-String: {crs_value}")
else:
    crs_value = None
    print("Keine CRS-Variable gefunden.")

CRS-Proj4-String: +proj=stere +lat_0=90 +lat_ts=90 +lon_0=10 +k_0=0.933012701892 +x_0=0 +y_0=0 +a=6370040 +b=6370040 +units=km +no_defs


## Schritt 2: Auswahl der Variable und des Zeitintervalls
Wähle die Variable aus der NetCDF-Datei, die du konvertieren möchtest (z.B. Temperatur, Niederschlag, etc.), und definiere den Zeitschritt, den du in eine GeoTIFF-Datei umwandeln möchtest.


In [18]:
import re

# Zeitschrittauswahl (manuelle Eingabe)
# Standard ist "all", aber kann auf spezifische Zeitschritte oder Bereiche angepasst werden.
time_step_input = "all"  # Beispiel: "0-10, 15, 20-25"
time_step_input = "0-3"

# Funktion zur Umwandlung der Eingabe in eine Liste von Zeitschritten
def parse_time_steps(input_str, ds):
    if input_str.strip().lower() == "all":
        return list(range(len(ds['time'])))
    else:
        time_steps = []
        ranges = input_str.split(",")
        for r in ranges:
            if "-" in r:
                start, end = map(int, r.split("-"))
                time_steps.extend(range(start, end + 1))
            else:
                time_steps.append(int(r))
        return time_steps

# Verarbeite die Eingabe der Zeitschritte
selected_time_steps = parse_time_steps(time_step_input, ds)

# Falls keine gültige Auswahl getroffen wurde
if not selected_time_steps:
    print("Keine gültige Auswahl getroffen. Vorgang wird abgebrochen.")
else:
    print(f"Ausgewählte Zeitschritte: {selected_time_steps}")


Ausgewählte Zeitschritte: [0, 1, 2, 3]


In [24]:
import rasterio
from rasterio.transform import from_origin
import pandas as pd
import re

# Funktion zur Bereinigung des Dateinamens
def clean_filename(filename):
    # Ersetze Punkte (.) im Dateinamen durch Unterstriche, außer der letzten für die Dateiendung
    cleaned_name = re.sub(r'\.', '_', filename[:filename.rfind('.')]) + filename[filename.rfind('.'):]
    return cleaned_name

# Metadaten (Transform) für die GeoTIFF-Erstellung
transform = from_origin(min(ds['lon'].values.flatten()), max(ds['lat'].values.flatten()), abs(ds['x'][1] - ds['x'][0]), abs(ds['y'][1] - ds['y'][0]))

# Funktion zur Konvertierung eines Zeitschritts in ein GeoTIFF
def export_to_geotiff(time_index, ds, crs_value, transform, nc_file_path):
    rr_data = ds['RR'].isel(time=time_index)  # Niederschlagsdaten für den Zeitschritt
    timestamp = pd.to_datetime(str(ds['time'].values[time_index]))  # Zeitstempel
    
    # Dateinamen generieren
    nc_filename = os.path.basename(nc_file_path).split('.')[0]
    cleaned_nc_filename = clean_filename(nc_filename)
    variable_name = 'RR'  # Variable, die exportiert wird
    output_filename = f"{cleaned_nc_filename}_{variable_name}_{timestamp.strftime('%Y%m%d_%H%M%S')}.tif"
    
    output_tiff = f'./output/{output_filename}'
    
    # Schreibe die Daten in eine GeoTIFF-Datei
    with rasterio.open(
        output_tiff, 'w',
        driver='GTiff',
        height=rr_data.shape[0],
        width=rr_data.shape[1],
        count=1,
        dtype=rr_data.dtype,
        crs=crs_value,  # Verwende die CRS-Information aus der Datei
        transform=transform
    ) as dst:
        dst.write(rr_data.values, 1)
    
    print(f"GeoTIFF für Zeitschritt {time_index} mit Zeitstempel {timestamp} wurde erfolgreich unter {output_tiff} gespeichert.")

# Iteriere über die ausgewählten Zeitschritte und exportiere sie als GeoTIFF
for time_index in selected_time_steps:
    export_to_geotiff(time_index, ds, crs_value, transform, nc_file_path)


GeoTIFF für Zeitschritt 0 mit Zeitstempel 2021-01-01 00:00:00 wurde erfolgreich unter ./output/YW_2017_RR_20210101_000000.tif gespeichert.
GeoTIFF für Zeitschritt 1 mit Zeitstempel 2021-01-01 00:05:00 wurde erfolgreich unter ./output/YW_2017_RR_20210101_000500.tif gespeichert.
GeoTIFF für Zeitschritt 2 mit Zeitstempel 2021-01-01 00:10:00 wurde erfolgreich unter ./output/YW_2017_RR_20210101_001000.tif gespeichert.
GeoTIFF für Zeitschritt 3 mit Zeitstempel 2021-01-01 00:15:00 wurde erfolgreich unter ./output/YW_2017_RR_20210101_001500.tif gespeichert.


## Schritt 3: Konvertierung in GeoTIFF
Nun konvertieren wir die extrahierten Rasterdaten in eine GeoTIFF-Datei. Dafür wird die Bibliothek `rasterio` verwendet. Wir müssen auch die Geoinformationen wie die Projektion (z.B. WGS84) und die Bounding Box bereitstellen.


In [ ]:
# Erstellen eines GeoTIFF-Dateipfads
output_tif = 'output_raster.tif'

# Beispielhafte affine Transformation und CRS (proj4 oder EPSG-Code)
transform = rasterio.transform.from_origin(west=0, north=0, xsize=0.1, ysize=0.1)  # Beispielhafte Parameter
crs = 'EPSG:4326'  # WGS84

# Öffnen einer neuen TIFF-Datei zum Schreiben
with rasterio.open(
    output_tif, 
    'w', 
    driver='GTiff', 
    height=data_at_time.shape[0], 
    width=data_at_time.shape[1], 
    count=1, 
    dtype=str(data_at_time.dtype), 
    crs=crs, 
    transform=transform
) as dst:
    # Schreibe die Daten in die TIFF-Datei
    dst.write(data_at_time.values, 1)

print(f'Datei {output_tif} erfolgreich erstellt.')


## Schritt 4: Konvertierung für alle Zeitschritte (optional)
Falls die NetCDF-Datei mehrere Zeitschritte enthält und du alle in separate TIFF-Dateien konvertieren möchtest, kannst du über die Zeitachse iterieren und für jeden Zeitschritt eine TIFF-Datei erzeugen.


In [ ]:
# Erstellen eines Ordners zum Speichern der GeoTIFFs
output_folder = 'output_tiffs'
os.makedirs(output_folder, exist_ok=True)

# Über die Zeitachsen iterieren
for i in range(len(ds.coords['time'])):
    # Extrahiere Daten für den aktuellen Zeitschritt
    data_at_time = data_var.isel(time=i)
    
    # Dateiname für den aktuellen Zeitschritt
    output_tif = os.path.join(output_folder, f'raster_time_{i}.tif')
    
    # GeoTIFF-Datei erstellen
    with rasterio.open(
        output_tif, 
        'w', 
        driver='GTiff', 
        height=data_at_time.shape[0], 
        width=data_at_time.shape[1], 
        count=1, 
        dtype=str(data_at_time.dtype), 
        crs=crs, 
        transform=transform
    ) as dst:
        dst.write(data_at_time.values, 1)

    print(f'Datei {output_tif} für Zeitschritt {i} erfolgreich erstellt.')


## Fazit
Das Skript ermöglicht die Konvertierung von NetCDF-Daten in GeoTIFF-Dateien, wobei Zeitschritte berücksichtigt werden. Diese Methode kann verwendet werden, um rasterbasierte Zeitreihenanalysen in einem georeferenzierten Format (TIFF) zu speichern und weiterzuverarbeiten.
